### Investigating the Importance of Low-carbon Growth in Sub-Saharan Africa

Different trajectories for economic and population growth in developing countries potentially have a very large impact on the climate budget. Currently (based on 2014 World Bank data), the US, China, and Europe combined account for 62% of global annual emissions but only 34% of global population. If the remainder of the world, especially sub-Saharan African countries, increase their per capita carbon emissions even modestly as a result of pursuing economic growth, the increase in total carbon emissions could be catastrophic. 

This notebook briefly explores the effect on global emissions if sub-Saharan African countries were to increase per capita carbon emissions by even a modest amount (to levels similar to those of Egypt or developing countries on other continents). The analysis suggests that investment in low-carbon development in sub-Saharan African countries is critical if their populations and economies continue to grow.  

In [2]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import folium
import requests
import io
import pysal as ps
from IPython.core.display import display, HTML
from IPython.display import IFrame

In [3]:
# Get CO2 emissions & population data by country
# https://data.worldbank.org/indicator/EN.ATM.CO2E.PC?view=map
emis = pd.read_csv('API_EN.ATM.CO2E.KT_DS2_en_csv_v2.csv', skiprows=2, header=1) # units: kt
# emis = pd.read_csv('API_EN.ATM.GHGT.KT.CE_DS2_en_csv_v2.csv', skiprows=2, header=1) # units: kt of CO2 equivalent
pop = pd.read_csv('API_SP.POP.TOTL_DS2_en_csv_v2.csv', skiprows=2, header=1) # units: millions
emisPerCap = pd.read_csv('API_EN.ATM.CO2E.PC_DS2_en_csv_v2.csv', skiprows=2, header=1) # metric tons per capita

In [4]:
# Get Country to Continent Crosswalk
url = 'https://raw.githubusercontent.com/zelima/country-continents/master/data/country-continents.csv'
urlData = requests.get(url).content
country2continent = pd.read_csv(io.StringIO(urlData.decode('utf-8')), keep_default_na=False)

In [5]:
years = [str(i+1960) for i in range(2014-1960+1)]

In [6]:
emis = pd.melt(emis, id_vars=['Country Name'], value_vars=years)
pop = pd.melt(pop, id_vars=['Country Name'], value_vars=years)
emisPerCap = pd.melt(emisPerCap, id_vars=['Country Name'], value_vars=years)

In [7]:
emisPop = emis.merge(pop, on=['Country Name', 'variable'], how='inner')
# Replace with common names
emisPop = emisPop.replace({'Russian Federation':'Russia', 
                          'Iran, Islamic Rep.':'Iran',
                          'Egypt, Arab Rep.':'Egypt', 
                          'Syrian Arab Republic':'Syria', 
                          'Congo, Dem. Rep.':'Democratic Republic of the Congo', 
                          'Venezuela, RB':'Venezuela', 
                          'Korea, Rep.':'South Korea', 
                          "Korea, Dem. People's Rep.":'North Korea', 
                          'Korea, Rep.':'South Korea', 
                          'Slovak Republic':'Slovakia',
                          'Yemen, Rep.': 'Yemen',
                          'Lao PDR': 'Laos',
                          'Kyrgyz Republic': 'Kyrgyzstan',
                          'Congo, Rep.': 'Republic of the Congo',
                          "Cote d'Ivoire": 'Ivory Coast'})
emisPop = emisPop.merge(country2continent, left_on='Country Name', right_on='Country')

In [8]:
emisPop.rename(columns={'Country Name': 'country', 'variable': 'year', 
                        'value_x':'co2', 'value_y':'pop', 'Continent': 'continent'}, inplace=True)
emisPop.drop('Country', axis=1, inplace=True)
# Convert from kt to metric tons
emisPop['co2'] = emisPop['co2']*1000
emisPop['co2perCap'] = emisPop['co2']/emisPop['pop']
emisPop = emisPop.loc[emisPop.co2perCap.notnull()]

#### Which continent has the lowest per capita CO2 emissions?

In [9]:
# Recalculate CO2 per capita at the continent level
emisByCont = emisPop.groupby(by=['continent', 'year']).sum()
emisByCont.reset_index(inplace=True)
emisByCont['co2perCap'] = emisByCont['co2']/emisByCont['pop']

In [10]:
emisByCont.loc[emisByCont['year']=='2014'].sort_values('co2perCap')

,continent,year,co2,pop,co2perCap
54,AF,2014,1.308411e+09,1.154939e+09,1.132884
329,SA,2014,1.228232e+09,4.120608e+08,2.980706
109,AS,2014,1.894219e+10,4.284612e+09,4.420982
164,EU,2014,5.397296e+09,7.383398e+08,7.310044
274,OC,2014,4.096662e+08,3.849511e+07,10.642034
219,NA,2014,6.451624e+09,5.620705e+08,11.478318


#### Which countries are the top emitters of CO2 in total?

In [39]:
emisPop.loc[(emisPop['year']=='2014')].sort_values('co2', ascending=False).head()

,country,year,co2,pop,continent,co2perCap,lessThan1
1869,China,2014,1.029193e+10,1.364270e+09,AS,7.543908,False
10339,United States,2014,5.254279e+09,3.185635e+08,NA,16.493666,False
4564,India,2014,2.238377e+09,1.293859e+09,AS,1.730000,False
8359,Russia,2014,1.705346e+09,1.438197e+08,EU,11.857528,False
5059,Japan,2014,1.214048e+09,1.272760e+08,AS,9.538706,False


#### Ratio of CO2 emissions and population in the United States, China, and Europe to global CO2 emissions and population 

In [58]:
US_China_EU = emisPop.loc[(emisPop['year']=='2014') & 
                           (emisPop['country'].isin(['United States', 'China']))][['co2', 'pop']].sum() + \
                               emisByCont.loc[(emisByCont['year']=='2014') & 
                                              (emisByCont['continent']=='EU')][['co2', 'pop']].sum()
world = emisPop.loc[emisPop['year']=='2014'][['co2', 'pop']].sum()

The US, China, and Europe combined account for 62% of global annual emissions but *only 34%* of global population.

In [59]:
US_China_EU / world

co2    0.620780
pop    0.336718
dtype: float64

#### How many countries in Africa have annual per capita CO2 emissions below 1 metric ton?

In [11]:
emisPop['lessThan1'] = emisPop['co2perCap'] < 1
lessThan1 = emisPop.loc[(emisPop['continent']=='AF') & 
                        (emisPop['year'] == '2014')].groupby('lessThan1')['pop'].agg(['sum','count'])
lessThan1['pctTotPop'] = lessThan1['sum'] / lessThan1['sum'].sum()

37 countries accounting for 76% of the population in Africa have annual per capita CO2 emissions below 1 metric ton.

In [12]:
lessThan1

,sum,count,pctTotPop
lessThan1,,,
False,2.747020e+08,14,0.23785
True,8.802369e+08,37,0.76215


#### Estimating the effect of an emissions growth scenario in Africa

How will CO2 emissions in Africa change if all countries with per capita emissions under 1 metric ton increased emissions to 2 metric tons per capita? Annual CO2 emissions of 2 metric tons would be roughly equivalent to the emissions level of Egypt.

In [24]:
lessThan1_pre = emisPop.loc[(emisPop.lessThan1 == True) & (emisPop.continent == 'AF') & (emisPop.year == '2014')]['co2'].sum()
lessThan1_post = emisPop.loc[(emisPop.lessThan1 == True) & (emisPop.continent == 'AF') & (emisPop.year == '2014')]['pop'].sum()*2

In [25]:
restOfAF = emisPop.loc[(emisPop.lessThan1 == False) & (emisPop.continent == 'AF') & (emisPop.year == '2014')]['co2'].sum()
allOfAF = emisPop.loc[(emisPop.continent == 'AF') & (emisPop.year == '2014')]['co2'].sum()

Emissions in sub-Saharan Africa would increase by *over 500%*.

In [27]:
(lessThan1_post - lessThan1_pre) / lessThan1_pre

5.709791627481168

Emissions in Africa overall would *more than double*.

In [22]:
((restOfAF + lessThan1_post) - allOfAF) / allOfAF

1.1449763904471537

#### Create map of CO2 emissions per capita

In [240]:
# Import countries shapefile
fp = 'shapefiles/ne_110m_admin_0_countries.shp'
data = gpd.read_file(fp)
# Replace with common names
data = data.replace({'Czechia': 'Czech Republic',
                             'Republic of Serbia': 'Serbia',
                             'Somoliland': 'Somalia'})

data = data.merge(emisPop.loc[emisPop['year'] == '2014'], how='inner', left_on='SUBUNIT', right_on='country', indicator=True)

co2Map = folium.Map(location=[30, 0], zoom_start=2)

In [241]:
def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 6, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(geo_str = gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
    return mapobj

The map shows that countries with per capita CO2 emissions between 0-1 metric tons are primarily located in sub-Saharan Africa. This region has both an opportunity and necessity for low-carbon growth, especially with some of the highest population growth rates in the world.

In [242]:
co2Map = add_choropleth(co2Map, data, 'country','co2perCap', classifier='Quantiles')
co2Map.save('co2Map.html')
IFrame("co2Map.html", width=1000, height=450)